In [2]:
# Display a list of packages that the current environment has.

# ! pip list

In [3]:
!pip install -qq requests beautifulsoup4 transformers sentence-transformers google-search-results

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 83.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 59.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 77.1 MB/s eta 0:00:00


In [5]:
import requests
from bs4 import BeautifulSoup
from transformers import pipeline
from sentence_transformers import SentenceTransformer, util
from serpapi import GoogleSearch
from datetime import datetime

In [6]:
# Load NLP Models
sentiment_analyzer = pipeline("sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment")
similarity_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/747 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [7]:
# API Keys

from google.colab import userdata

# https://moz.com/api/dashboard
MOZ_API_KEY = userdata.get('MOZ_API_KEY')
MOZ_API_URL = "https://lsapi.seomoz.com/v2/url_metrics"

# https://console.cloud.google.com/apis/credentials?inv=1&invt=AboofA&project=cs667-449902&supportedpurview=project
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
GOOGLE_FACT_CHECK_URL = "https://factchecktools.googleapis.com/v1alpha1/claims:search"
GOOGLE_SAFE_BROWSING_URL = "https://safebrowsing.googleapis.com/v4/threatMatches:find"

# https://user.whoisxmlapi.com/products
WHOIS_API_KEY = userdata.get('WHOIS_API_KEY')
WHOIS_URL = "https://www.whoisxmlapi.com/whoisserver/WhoisService"

# https://serpapi.com/dashboard
SERP_API_KEY = userdata.get('SERP_API_KEY')

In [8]:
# Exceeded 50 requests from MOZ Free Plan
'''
def get_domain_trust_score(url):
    """Check the domain's credibility using Moz API."""
    headers = {"Authorization": f"Bearer {MOZ_API_KEY}"}
    payload = {"targets": [url]}  # Use the function argument, not a hardcoded value
    try:
        response = requests.post(MOZ_API_URL, headers=headers, json=payload)  # Send data as JSON
        if response.status_code == 200:
            data = response.json()
            print("Moz API Response:", data)  # Debugging Step
            if "results" in data and len(data["results"]) > 0:
                return min(data["results"][0].get("domain_authority", 0), 100)
        print(f"Error {response.status_code}: {response.text}")  # Debugging
        return 1  # Default low trust if request fails
    except requests.exceptions.RequestException as e:
        print(f"Request Error: {e}")
        return 1  # Return default low trust if request fails
'''


# Use alternative APIs:
def get_google_safety_score(url):
    """Check Google Safe Browsing API for security risks."""
    payload = {
        "client": {"clientId": "your-client-id", "clientVersion": "1.0"},
        "threatInfo": {
            "threatTypes": ["MALWARE", "SOCIAL_ENGINEERING", "UNWANTED_SOFTWARE", "POTENTIALLY_HARMFUL_APPLICATION"],
            "platformTypes": ["ANY_PLATFORM"],
            "threatEntryTypes": ["URL"],
            "threatEntries": [{"url": url}]
        }
    }
    try:
        response = requests.post(f"{GOOGLE_SAFE_BROWSING_URL}?key={GOOGLE_API_KEY}", json=payload)
        data = response.json()
        if "matches" in data:
            return 1  # Unsafe domain → Very low trust score
        return 100  # Safe domain → High trust score
    except requests.exceptions.RequestException as e:
        print(f"Safe Browsing API Error: {e}")
        return 50  # Neutral trust if request fails

def get_domain_age_score(url):
    """Check domain age using WHOIS API (Older domains are more trustworthy)."""
    try:
        domain = url.split("//")[-1].split("/")[0]  # Extract domain name
        whois_url = f"{WHOIS_URL}?apiKey={WHOIS_API_KEY}&domainName={domain}&outputFormat=json"

        response = requests.get(whois_url)
        data = response.json()

        if "WhoisRecord" in data and "createdDate" in data["WhoisRecord"]:
            created_date = data["WhoisRecord"]["createdDate"]  # e.g., "1997-03-03T05:00:00Z"
            domain_year = int(created_date.split("-")[0])  # Extract the year
            current_year = datetime.now().year
            domain_age = current_year - domain_year

            # Scale domain age score (0 to 100)
            return min(domain_age * 10, 100)  # 10 years or more = max score

        return 50  # Default mid-trust if no data found

    except requests.exceptions.RequestException as e:
        print(f"API Error: {e}")
        return 50  # Return default mid-trust if request fails

def get_google_search_popularity(url):
    """Check Google search popularity using SerpAPI (Higher presence = higher trust)."""
    try:
        domain = url.split("//")[-1].split("/")[0]  # Extract domain name

        # Call SerpAPI to get Google search results
        params = {
            "q": f"site:{domain}",
            "engine": "google",
            "api_key": SERP_API_KEY
        }
        search = GoogleSearch(params)
        results = search.get_dict()

        # Extract search result count
        organic_results = results.get("organic_results", [])
        popularity_score = min(len(organic_results) * 10, 100)  # Normalize (0-100)

        return popularity_score
    except Exception as e:
        print(f"Error: {e}")
        return 50  # Default mid-trust if API request fails

def get_domain_trust_score(url):
    """Combine Google Safe Browsing, Domain Age, and Search Popularity to compute domain trust."""
    safety_score = get_google_safety_score(url)
    domain_age_score = get_domain_age_score(url)
    popularity_score = get_google_search_popularity(url)

    final_trust_score = (safety_score * 0.4) + (domain_age_score * 0.3) + (popularity_score * 0.3)
    return round(final_trust_score, 2)


# comment out to reserve api requests
# Test cases:
'''
urls = [
    "https://www.bhtp.com/blog/when-safe-to-travel-with-newborn/",
    "https://www.reddit.com/",
    "http://malicious-site-example.com",
    "https://www.youtube.com",
    "https://www.apple.com"
]

for url in urls:
    print(f"Testing: {url}")
    print(f"Trust Score: {get_domain_trust_score(url)}\n")
'''
# Outputs:
'''
Testing: https://www.bhtp.com/blog/when-safe-to-travel-with-newborn/
Trust Score: 100.00

Testing: https://www.reddit.com/
Trust Score: 100.00

Testing: http://malicious-site-example.com
Trust Score: 55.00

Testing: https://www.youtube.com
Trust Score: 100.00

Testing: https://www.apple.com
Trust Score: 100.00
'''

'\nTesting: https://www.bhtp.com/blog/when-safe-to-travel-with-newborn/\nTrust Score: 100.00\n\nTesting: https://www.reddit.com/\nTrust Score: 100.00\n\nTesting: http://malicious-site-example.com\nTrust Score: 55.00\n\nTesting: https://www.youtube.com\nTrust Score: 100.00\n\nTesting: https://www.apple.com\nTrust Score: 100.00\n'

In [9]:
def get_content_relevance_score(url, query):
    """Check content relevance using NLP similarity score."""
    page = requests.get(url)
    soup = BeautifulSoup(page.text, "html.parser")
    text = " ".join([p.get_text() for p in soup.find_all("p")])[:2000]
    embeddings1 = similarity_model.encode(query, convert_to_tensor=True)
    embeddings2 = similarity_model.encode(text, convert_to_tensor=True)
    similarity_score = util.pytorch_cos_sim(embeddings1, embeddings2).item()
    return round(similarity_score*100, 2)


# Test cases:
'''
print(get_content_relevance_score("https://en.wikipedia.org/wiki/Bitcoin","What is bitcoin?"))
print(get_content_relevance_score("https://www.schwab.com/stocks/understand-stocks","What is cryptocurrency?"))
'''
# Outputs:
'''
73.5
2.6
'''

'\n73.5\n2.6\n'

In [10]:
def get_fact_check_score(query):
    """
    Check if a claim is factually correct using Google Fact Check API.

    Returns:
    - 100 → Fully True
    - 0 → Fully False
    - 25 → No Fact-Check Available
    """
    params = {"query": query, "key": GOOGLE_API_KEY}
    response = requests.get(GOOGLE_FACT_CHECK_URL, params=params)

    if response.status_code == 200:
        data = response.json()

        if "claims" not in data or len(data["claims"]) == 0:
            return 25  # No fact-check available

        fact_check_scores = []  # Store fact-check results

        for claim in data["claims"]:
            reviews = claim.get("claimReview", [])
            for review in reviews:
                rating = review.get("textualRating", "").lower()

                # Assign binary fact-check scores
                if "true" in rating:
                    fact_check_scores.append(100)
                elif "false" in rating:
                    fact_check_scores.append(0)

        # If multiple fact-checks exist, return the most confident result
        return max(fact_check_scores) if fact_check_scores else 25

    print(f"API Error {response.status_code}: {response.text}")  # Debugging API errors
    return 25  # Default to no fact-check found


# Test cases:
'''
print(get_fact_check_score("The Earth is flat"))  # Expected: 0
print(get_fact_check_score("Joe Biden won the 2020 US Presidential Election"))  # Expected: 100
print(get_fact_check_score("Sugar is sweet"))  # Expected: 25 (No fact-check)
'''
# Outputs:
'''
0
100
25
'''

'\n0\n100\n25\n'

In [11]:
def get_bias_score(url):
    """Analyze text bias using Hugging Face sentiment model."""
    try:
        if not url.strip():
            return "Error: URL is empty"

        # Fetch page content
        response = requests.get(url, timeout=5, headers={"User-Agent": "Mozilla/5.0"})
        if response.status_code != 200:
            return f"Error: Unable to access URL (HTTP {response.status_code})"

        soup = BeautifulSoup(response.text, "html.parser")
        text = soup.get_text()

        if len(text) < 100:
            return "Error: Not enough content to analyze"

        # **Fix for Transformer Token Limit (Max 512 Tokens)**
        text = text[:2000]  # Extract the first 1024 characters
        words = text.split()[:500]  # Approximate token limit (~1.5x characters per token)
        truncated_text = " ".join(words)

        # Run sentiment analysis
        sentiment = sentiment_analyzer(truncated_text)[0]
        score = sentiment["score"]
        label = sentiment["label"]

        # Convert negative sentiment into bias score
        if label == "NEGATIVE":
            bias_score = 60 + ((1 - score) * 40)  # Bias starts at 60%, scales up to 100%
        elif label == "POSITIVE":
            bias_score = 40 - (score * 40)  # Bias starts at 40%, scales down to 0%
        else:  # Neutral
            bias_score = 50  # Set a mid-point for neutrality

        return round(score*100, 2)  # Reward neutral/positive content

    except requests.exceptions.RequestException as e:
        return f"Error: {str(e)}"


# Test cases:
'''
test_urls = [
    "https://www.npr.org/sections/news/",  # Left-Leaning
    "https://www.bbc.com/news",  # Neutral
    "https://www.foxnews.com/politics/latest-debate",  # Right-Leaning
    "",  # Empty URL
    "https://invalid-url-example.com",  # Non-Accessible Page
]

for url in test_urls:
    print(f"Testing: {url}")
    print(f"Bias Score: {get_bias_score(url)}\n")
'''
# Outputs:
'''
Testing: https://www.npr.org/sections/news/
Bias Score: 85.9

Testing: https://www.bbc.com/news
Bias Score: 59.7

Testing: https://www.foxnews.com/politics/latest-debate
Bias Score: Error: Unable to access URL (HTTP 404)

Testing:
Bias Score: Error: URL is empty

Testing: https://invalid-url-example.com
Bias Score: Error: HTTPSConnectionPool(host='invalid-url-example.com', port=443): Max retries exceeded with url: / (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7d198562b310>: Failed to resolve 'invalid-url-example.com' ([Errno -2] Name or service not known)"))
'''

'\nTesting: https://www.npr.org/sections/news/\nBias Score: 85.9\n\nTesting: https://www.bbc.com/news\nBias Score: 59.7\n\nTesting: https://www.foxnews.com/politics/latest-debate\nBias Score: Error: Unable to access URL (HTTP 404)\n\nTesting: \nBias Score: Error: URL is empty\n\nTesting: https://invalid-url-example.com\nBias Score: Error: HTTPSConnectionPool(host=\'invalid-url-example.com\', port=443): Max retries exceeded with url: / (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7d198562b310>: Failed to resolve \'invalid-url-example.com\' ([Errno -2] Name or service not known)"))\n'

In [12]:
def get_citation_score(query):
    """Use SerpAPI to find scholarly references related to the topic. Score ranges from 0-100."""
    params = {
        "q": query,
        "engine": "google_scholar",
        "api_key": SERP_API_KEY
    }

    search = GoogleSearch(params)
    results = search.get_dict()  # ✅ Correct method

    # Check if scholarly references exist
    if "organic_results" in results:
        num_results = len(results["organic_results"])
        return min(num_results * 10, 100)  # Each result adds 10 points, max 100
    return 0  # No citations found


# Test cases:
'''
print(get_citation_score("Deep Learning in Medical Imaging"))
print(get_citation_score("Best travel destinations 2024"))
'''
# Outputs:
'''
100
100
'''

'\n100\n100\n'

In [13]:
WEIGHTS = {
    "domain_trust": 0.275,       # 27.5%
    "content_relevance": 0.275,  # 27.5%
    "fact_check": 0.275,         # 27.5%
    "bias": 0.125,               # 12.5%
    "citation": 0.05             # 05.0%
}

In [20]:
def validate_url(prompt, url):
    """
    Main function to rate a source based on multiple credibility factors.

    Each individual score range from 0 to 100.

    Then, weight is applied according to WEIGHTS dict above to calculate the final score.
    """
    domain_trust_score = get_domain_trust_score(url)
    weighted_domain_trust_score = round(domain_trust_score * WEIGHTS['domain_trust'], 2)

    content_relevance_score = get_content_relevance_score(url, prompt)
    weighted_content_relevance_score = round(content_relevance_score * WEIGHTS['content_relevance'], 2)

    fact_check_score = get_fact_check_score(prompt)
    weighted_fact_check_score = round(fact_check_score * WEIGHTS['fact_check'], 2)

    bias_score = get_bias_score(url)
    weighted_bias_score = round(bias_score * WEIGHTS['bias'], 2)

    citation_score = get_citation_score(prompt)
    weighted_citation_score = round(citation_score * WEIGHTS['citation'], 2)

    final_score = round(
        weighted_domain_trust_score +
        weighted_content_relevance_score +
        weighted_fact_check_score +
        weighted_bias_score +
        weighted_citation_score
    , 2)

    return {
        "url": url,
        "domain_trust": domain_trust_score,
        "w_domain_trust": weighted_domain_trust_score,
        "relevance_score": content_relevance_score,
        "w_relevance_score": weighted_content_relevance_score,
        "fact_check": fact_check_score,
        "w_fact_check": weighted_fact_check_score,
        "bias_score": bias_score,
        "w_bias_score": weighted_bias_score,
        "citation_score": citation_score,
        "w_citation_score": weighted_citation_score,
        "final_score": final_score
    }

# Test case:
'''
url = "https://www.bhtp.com/blog/when-safe-to-travel-with-newborn/"
prompt = "I have just been on an international flight, can i come back home to hold my 1 month old newborn?"
result = validate_url(prompt, url)
print(result)
'''
# Output:
'''
{'url': 'https://www.bhtp.com/blog/when-safe-to-travel-with-newborn/',
 'domain_trust': 100.0, 'w_domain_trust': 27.5,
 'relevance_score': 56.17, 'w_relevance_score': 15.45,
 'fact_check': 25, 'w_fact_check': 6.88,
 'bias_score': 65.36, 'w_bias_score': 8.17,
 'citation_score': 100, 'w_citation_score': 5.0,
 'final_score': 63.0}
'''

"\n{'url': 'https://www.bhtp.com/blog/when-safe-to-travel-with-newborn/', \n 'domain_trust': 100.0, 'w_domain_trust': 27.5, \n 'relevance_score': 56.17, 'w_relevance_score': 15.45, \n 'fact_check': 25, 'w_fact_check': 6.88, \n 'bias_score': 65.36, 'w_bias_score': 8.17, \n 'citation_score': 100, 'w_citation_score': 5.0, \n 'final_score': 63.0}\n"

In [21]:
def get_explanation():
  explanation = "___" # TEMPORARY
  return explanation

In [22]:
def credibility_score(prompt, url):
  scores = validate_url(prompt, url)
  credibility_score = scores['final_score']
  explanation = get_explanation()
  result = {'score': round(credibility_score, 2), 'explanation': explanation}
  return result

In [26]:
# RESULTS:
user_prompt = "I have just been on an international flight, can i come back home to hold my 1 month old newborn?"
url_ref = "https://www.bhtp.com/blog/when-safe-to-travel-with-newborn/"

score = credibility_score(user_prompt, url_ref)
print(score)

{'score': 63.0, 'explanation': '___'}


<hr>
<hr>

 **Extra**

In [27]:
import json

def get_json():
  data = score
  return json.dumps(data)

json_object = get_json()
print(json_object)

{"score": 63.0, "explanation": "___"}


In [28]:
# write to json file:
'''
with open("result.json", "w") as json_file:
  json_file.write(json_object)
'''

'\nwith open("result.json", "w") as json_file:\n  json_file.write(json_object)\n'

In [29]:
# Download the json file:
'''
from google.colab import files
files.download("result.json")
'''

'\nfrom google.colab import files\nfiles.download("result.json")\n'

<hr>
<hr>